# Numerical Linear Algebra Lab

## Iterative Solvers: Update Jacobi to Gauss-Seidel

We learned in the lecture about the Jacobi interation method for solving $\mathbf{x}$ in the linear equation $A \mathbf{x} = \mathbf{b}$.
The algorithm states that by applying the update rule:
\begin{align}
  x_i^{(k+1)} = \frac{1}{a_{ii}} \Big(b_i - \sum_{j \neq i} a_{ij} x_j^{(k)}\Big),
\end{align}
$x_i^{(k+1)}$ will converge to the solution $\mathbf{x}$.

We implemeneted and test it in python:

In [ ]:
def jacobi_iteration(A, b, max_iter=1000, tol=1e-8):
    """
    Solve A x = b using Jacobi iteration.
    A is assumed to be square with non-zero diagonal.
    """
    n = A.shape[0]
    x = np.zeros(n)
    
    for k in range(max_iter):
        x_old = np.copy(x)
        for i in range(n):
            # Sum over off-diagonal terms
            s = 0.0
            for j in range(n):
                if j != i:
                    s += A[i,j] * x_old[j]
            x[i] = (b[i] - s) / A[i, i]
        
        # Check for convergence
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Jacobi converged in {k+1} iterations.")
            return x
    
    print("Jacobi did not fully converge within max_iter.")
    return x

In [ ]:
import numpy as np

A = np.array([[4.0, -1.0, 0.0],
              [-1.0, 4.0, -1.0],
              [0.0, -1.0, 4.0]], dtype=float)
b = np.array([6.0, 6.0, 6.0], dtype=float)

x_jacobi = jacobi_iteration(A, b)

print("Jacobi solution:", x_jacobi)
print("Direct solve comparison:", np.linalg.solve(A, b))

For the first hands-on, please update the above Jacobi algorithm and turn it into a working Gauss Seidel algorithm.

Recalling from the lecture that the update step in the Gauss Seidel algorithm is:
\begin{align}
     x_i^{(k+1)} = \frac{1}{a_{ii}} \Big( b_i - \sum_{j < i} a_{ij} x_j^{(k+1)} - \sum_{j > i} a_{ij} x_j^{(k)} \Big).
\end{align}
Note that $\mathbf{x}^{(k+1)}$ is already partially updated (for $j < i$).

In [ ]:
def gauss_seidel_iteration(A, b, max_iter=1000, tol=1e-8):
    """
    Solve A x = b using the Gauss-Seidel iteration.
    A is assumed to be square with non-zero diagonal.
    """
    n = A.shape[0]
    x = np.zeros(n)

    for k in range(max_iter):
        x_old = np.copy(x)
        for i in range(n):
            # Sum over known terms
            s1 = 0.0
            for j in range(i):
                s1 += A[i, j] * x[j]  # x[j] is updated
            s2 = 0.0
            for j in range(i+1, n):
                s2 += A[i, j] * x_old[j]  # x_old[j] is from previous iteration
            x[i] = (b[i] - s1 - s2) / A[i, i]

        # Check convergence
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Gauss-Seidel converged in {k+1} iterations.")
            return x

    print("Gauss-Seidel did not fully converge within max_iter.")
    return x

In [ ]:
x_gs = gauss_seidel_iteration(A, b)

print("Jacobi solution:", x_jacobi)
print("Gauss-Seidel solution:", x_gs)
print("Direct solve comparison:", np.linalg.solve(A, b))

## Coupled Harmonic Oscillators

Harmonic oscillators are a classic problem in physics, often forming the basis of more complex vibrational analyses.
In this part, you will:
1. Derive or reference the analytical solution for two coupled oscillators.
2. Numerically solve the same system (using an eigenvalue approach).
3. Generalize to $n$ (and even $n \times n$) coupled oscillators, visualizing the mode shapes.

### Two Coupled Oscillators--Analytical Solution

Consider two masses $m$ connected by three springs (constant $k$), arranged in a line and connected to two walls:
```
|--k--[m]--k--[m]--k--|
```

If each mass can move only horizontally, the equations of motion form a $2 \times 2$ eigenvalue problem.

Let:
* $x_1(t)$ be the horizontal displacement of **Mass 1** from its equilibrium position.
* $x_2(t)$ be the horizontal displacement of **Mass 2**.

We assume **small oscillations**, so Hooke’s law applies linearly.

* **Mass 1** experiences:
  1. A restoring force $-k \,x_1$ from the left wall spring.
  2. A coupling force from the middle spring:
     if $x_2 > x_1$, that spring pulls Mass 1 to the right;
     if $x_1 > x_2$, it pulls Mass 1 to the left.
     The net contribution is $-k (x_1 - x_2)$.

  Summing forces (Newton's second law) gives:
  \begin{align}
  m \, \ddot{x}_1 = -k\,x_1 - k\,(x_1 - x_2).
  \end{align}

* **Mass 2** experiences:
  1. A restoring force $-k\,x_2$ from the right wall spring.
  2. The coupling force from the middle spring: $-k(x_2 - x_1)$.

  Hence,
  \begin{align}
  m \, \ddot{x}_2 = -k\,x_2 - k\,(x_2 - x_1).
  \end{align}



Rewrite each equation:
\begin{align}
\begin{cases}
m\,\ddot{x}_1 + 2k\,x_1 -  k\,x_2 = 0,\\
m\,\ddot{x}_2 -  k\,x_1 + 2k\,x_2 = 0.
\end{cases}
\end{align}

We can write $\mathbf{x} = \begin{pmatrix}x_1 \\ x_2\end{pmatrix}$ and express the system as:
\begin{align}
m \,\ddot{\mathbf{x}} + K\,\mathbf{x} = \mathbf{0},
\end{align}
where
\begin{align}
m \,\ddot{\mathbf{x}} = m \begin{pmatrix}\ddot{x}_1 \\[6pt] \ddot{x}_2\end{pmatrix}, \quad
K = \begin{pmatrix}
2k & -k \\
-k & 2k
\end{pmatrix}.
\end{align}

Equivalently,
\begin{align}
\ddot{\mathbf{x}} + \frac{1}{m}\,K \,\mathbf{x} = \mathbf{0}.
\end{align}
This is a **second-order linear system** describing small oscillations.

We look for solutions of the form
\begin{align}
\mathbf{x}(t) = \mathbf{x}(0)\, e^{\,i\,\omega\,t},
\end{align}
where $\mathbf{x}(0)$ is the initial condition and $\omega$ is the (angular) oscillation frequency.

Plugging into $m\,\ddot{\mathbf{x}} + K\,\mathbf{x} = 0$ gives:
\begin{align}
-\,m\,\omega^2 \,\mathbf{X} + K\,\mathbf{X} = \mathbf{0}
\quad \Longrightarrow \quad
\left(K - m\,\omega^2 I\right)\,\mathbf{X} = \mathbf{0}.
\end{align}
Nontrivial solutions exist only if
\begin{align}
\det(K - m\,\omega^2 I) = 0,
\end{align}
which is the **eigenvalue problem** for $\omega^2$.

Explicitly, $K - m\,\omega^2 I$ is:
\begin{align}
\begin{pmatrix}
2k - m\,\omega^2 & -k \\
-k & 2k - m\,\omega^2
\end{pmatrix}.
\end{align}

The determinant is:
\begin{align}
(2k - m\,\omega^2)(2k - m\,\omega^2) - (-k)(-k)
= (2k - m\,\omega^2)^2 - k^2.
\end{align}

Setting this to zero results 
\begin{align}
2k - m\,\omega^2 = \pm\,k.
\end{align}

Hence, we get **two** solutions for $\omega^2$:
1. **$\omega_+^2$**: taking the $+$ sign:
   \begin{align}
   2k - m\,\omega_+^2 = k \quad \Longrightarrow \quad \omega_+^2 = \frac{k}{m} \quad \Longrightarrow \quad \omega_1 = \sqrt{\frac{k}{m}}.
   \end{align}
2. **$\omega_-^2$**: taking the $-$ sign:
   \begin{align}
   2k - m\,\omega_-^2 = -\,k \quad \Longrightarrow \quad \omega_-^2 = \frac{3k}{m} \quad \Longrightarrow \quad \omega_2 = \sqrt{\frac{3k}{m}}.
   \end{align}

For each of the normal modes:

* **Lower Frequency** $\omega_+ = \sqrt{k/m}$:
  Plug $\omega_+^2 = k/m$ back into $(K - m\,\omega_+^2 I)\mathbf{X} = 0$.
  For instance,
  \begin{align}
    \begin{pmatrix}
    2k - k & -k \\
    -k & 2k - k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \begin{pmatrix}
    k & -k \\
    -k & k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \mathbf{0}.
  \end{align}
  This implies $x_1 = x_2$.
  Physically, the **in-phase** mode has both masses moving together.

* **Higher Frequency** $\omega_- = \sqrt{3k/m}$:
  \begin{align}
    \begin{pmatrix}
    2k - 3k & -k \\
    -k & 2k - 3k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \begin{pmatrix}
    -k & -k \\
    -k & -k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \mathbf{0}.
  \end{align}
  This yields $x_1 = -x_2$.
  Physically, the **out-of-phase** mode has the two masses moving in opposite directions.

We can compute the position of these coupled oscillators according to the analytical solutions.

In [ ]:
# Physical constants
m = 1.0     # mass
k = 1.0     # spring constant

# Frequencies for two normal modes
omegap = np.sqrt(k/m)      # in-phase
omegam = np.sqrt(3*k/m)    # out-of-phase

# Initial conditions
x1_0 = 0
x2_0 = 0.5

# The analytical solution:
def X_analytic(t):
    xp_0 = (x1_0 + x2_0) / 2
    xm_0 = (x1_0 - x2_0) / 2

    xp = xp_0 * np.cos(omegap * t)
    xm = xm_0 * np.cos(omegam * t)

    return np.array([xp + xm, xp - xm])

Plot multiple frames:

In [ ]:
from pathlib import Path

from tqdm import tqdm

import matplotlib.pyplot    as plt
import matplotlib.animation as animation

def mkplots(X, t_max=10, n_frames=201):
    T = np.linspace(0, t_max, n_frames)

    Path("plots").mkdir(parents=True, exist_ok=True)
    Xs = []
    for t in T:
        Xs.append(X(t))
    Xs = np.array(Xs)

    for i, t in tqdm(enumerate(T)):
        plt.axhline(y=T[i], color='k', zorder=-1)
        for j in range(Xs.shape[-1]):
            plt.plot(j+1+Xs[:i+1,j], T[:i+1])
            plt.scatter(j+1+Xs[i,j], T[i])
        plt.xlim(0, Xs.shape[-1]+1)        
        plt.xlabel(r'$x_i$')
        plt.ylabel(r'$t$')
        plt.savefig(f"plots/{i:04}.png")
        plt.close()

In [ ]:
! rm -rf plots/
mkplots(X_analytic)

Can combine them into a movie using `ffmpeg`:

In [ ]:
# This installs `ffmpeg` if it is not available

! if [ ! $(which ffmpeg) ]; then apt update && apt install ffmpeg; fi

In [ ]:
# This convert all image files in "plots/" into "movie.mpg"

! rm -rf movie-2.mpg && ffmpeg -i plots/%04d.png -qmax 2 movie-2.mpg

### Two Coupled Oscillators--Semi-Analytical/Numerical Solution

Instead of solving the coupled oscillator problem analytically, we can at least solve the eigenvalue part of the problem numerically.

In [ ]:
# HANDSON: Step 1. rewrite the analytical solution in matrix form

# Physical constants
m = 1.0     # mass
k = 1.0     # spring constant

# Frequencies for two normal modes
Omega = np.array([np.sqrt(k/m), np.sqrt(3*k/m)]) # this should become a numpy array

# Initial conditions
X0 = np.array([0, 0.5]) # this should become a numpy array
M0 = np.array([
    [1/2,  1/2],
    [1/2, -1/2],
]) @ X0 # apply an transformation to rewrite the transformation in terms of eigenvectors

# The analytical solution in matrix notation:
def X_matrix(t): # closure on `M0` and `Omega`
    M = M0 * np.cos(Omega * t)
    return np.array([
        [1,  1],
        [1, -1],
    ]) @ M # apply an inverse transformation to rewrite the modes in terms of x1 and x2

In [ ]:
print(X_analytic(1))
print(X_matrix(1))

In [ ]:
# HANDSON: Step 2. Replace the manual solutions of eigenvalues Omega and the transform by calling `np.linalg.eig()`

K = np.array([
    [2*k/m, - k/m],
    [- k/m, 2*k/m],
])

# Initial conditions
X0 = np.array([0, 0.5]) # this should become a numpy array

# The semi-analytical solution in matrix notation:
Omega2, V = np.linalg.eig(K)
Omega     = np.sqrt(Omega2)
M0        = V.T @ X0

def X_numeric(t): # closure on `M0` and `Omega`
    return V @ (M0 * np.cos(Omega * t))

In [ ]:
print(X_analytic(1))
print(X_numeric(1))

In [ ]:
# HANDSON: Step 3. Generalize the solution to work for arbitrary number of coupled oscillators

K = np.array([
    [2*k/m, - k/m,   0  ],
    [- k/m, 2*k/m, - k/m],
    [  0,   - k/m, 2*k/m],
])

# Initial conditions
X0 = np.array([0,0,0.5]) #should become a numpy array

# The semi-analytical solution in matrix notation:
Omega2, V = np.linalg.eig(K)
Omega     = np.sqrt(Omega2)
M0        = V.T @ X0

In [ ]:
! rm -rf plots/
mkplots(X_numeric)
! rm -rf movie-3.mpg && ffmpeg -i plots/%04d.png -qmax 2 movie-3.mpg

In [ ]:
N = 4
K = (np.diag(np.repeat(2*k/m, N  )    ) +
     np.diag(np.repeat(- k/m, N-1), +1) +
     np.diag(np.repeat(- k/m, N-1), -1) )

print(K)

# Initial conditions
X0 = np.concatenate([np.zeros(N-1), [0.5]])

# The semi-analytical solution in matrix notation:
Omega2, V = np.linalg.eig(K)
Omega     = np.sqrt(Omega2)
M0        = V.T @ X0

In [ ]:
! rm -rf plots/
mkplots(X_numeric)
! rm -rf movie-4.mpg && ffmpeg -i plots/%04d.png -qmax 2 movie-4.mpg

In [ ]:
N = 16
K = (np.diag(np.repeat(2*k/m, N  )    ) +
     np.diag(np.repeat(- k/m, N-1), +1) +
     np.diag(np.repeat(- k/m, N-1), -1) )

print(K)

# Initial conditions
X0 = np.concatenate([np.zeros(N-1), [0.5]])

# The semi-analytical solution in matrix notation:
Omega2, V = np.linalg.eig(K)
Omega     = np.sqrt(Omega2)
M0        = V.T @ X0

In [ ]:
! rm -rf plots/
mkplots(X_numeric, t_max=100, n_frames=2001)
! rm -rf movie-16.mpg && ffmpeg -i plots/%04d.png -qmax 2 movie-16.mpg